In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [179]:
df = pd.read_csv('/content/drive/MyDrive/A2_Data.csv')
key = (df['Unnamed: 0'].tolist())
df = df.drop(columns=['Unnamed: 0'])
df["id"] = key

In [180]:
import ast
def fetch_successful_link(links):
    for link in links:
        try:
            response = requests.get(link)
            if response.status_code == 200:
                return link
        except Exception as e:
            pass  # Ignore any exceptions and continue to the next link
    return None

# Convert the string representation to a list using ast.literal_eval
# df['Image'] = df['Image'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# # Extract the first link from the 'Image' column
# df['First_Link'] = df['Image'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

# Convert the string representation to a list using ast.literal_eval
df['Image'] = df['Image'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['image'] = df['Image'].apply(lambda x: fetch_successful_link(x) if isinstance(x, list) and len(x) > 0 else x)

In [197]:
df = df.dropna(subset=['image'])

In [198]:
df

,Image,Review Text,id,image
0,[https://images-na.ssl-images-amazon.com/image...,Loving these vintage springs on my vintage str...,3452,https://images-na.ssl-images-amazon.com/images...
1,[https://images-na.ssl-images-amazon.com/image...,Works great as a guitar bench mat. Not rugged ...,1205,https://images-na.ssl-images-amazon.com/images...
2,[https://images-na.ssl-images-amazon.com/image...,We use these for everything from our acoustic ...,1708,https://images-na.ssl-images-amazon.com/images...
3,[https://images-na.ssl-images-amazon.com/image...,Great price and good quality. It didn't quite...,2078,https://images-na.ssl-images-amazon.com/images...
4,[https://images-na.ssl-images-amazon.com/image...,I bought this bass to split time as my primary...,801,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...
995,[https://images-na.ssl-images-amazon.com/image...,Extremely impressed with this kit.,1265,https://images-na.ssl-images-amazon.com/images...
996,[https://images-na.ssl-images-amazon.com/image...,This is a great stereo reverb with plenty of c...,1882,https://images-na.ssl-images-amazon.com/images...
997,[https://images-na.ssl-images-amazon.com/image...,I really like the simplicity of this bridge. I...,1547,https://images-na.ssl-images-amazon.com/images...
998,[https://images-na.ssl-images-amazon.com/image...,"Great Product, but there is no warranty in the...",1004,https://images-na.ssl-images-amazon.com/images...


In [194]:
desired_link = df.loc[df['id'] == 2088, 'image']

In [195]:
desired_link

860    None
Name: image, dtype: object

Image feature extraction

In [199]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

# Assuming your dataset is stored in a CSV file named 'data.csv'
# df = pd.read_csv('data.csv')
# Initialize dictionaries to store images and reviews
def fetch_image(url, id):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        img = Image.open(BytesIO(response.content))
        return img
    except Exception as e:
        print(f"Error fetching image from {id}: {e}")
        return None

# Dictionary to store images and reviews
image_dict = {}
review_dict = {}

# Iterate over DataFrame rows
for index, row in df.iterrows():
    image_id = row['id']
    # image_url = (row['Image'])[2:-2]
    image_url = (row['image'])
    review_text = row['Review Text']
    # print(image_url)
    # Fetch image and store in dictionary
    image = fetch_image(image_url, image_id)
    image_dict[image_id] = image

    # Store review text in dictionary
    review_dict[image_id] = review_text

# Accessing images and reviews
# for image_id, image in image_dict.items():
#     review_text = review_dict[image_id]
#     print(f"Image ID: {image_id}")
#     display(image)  # Display image
#     print(f"Review Text: {review_text}\n")

# Now, images_dict contains images and reviews_dict contains review texts
# You can access them using image ID as keys


In [200]:
from PIL import Image, ImageEnhance
import numpy as np
import cv2
import random

def preprocess_image(image):
    # Check if the image is grayscale or RGB
    if image:
        is_rgb = image.mode == 'RGB'

        # Randomly flip the image horizontally
        if is_rgb and random.choice([True, False]):
            image = image.transpose(method=Image.FLIP_LEFT_RIGHT)

        # Randomly adjust brightness
        brightness_factor = random.uniform(0.5, 1.5)
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(brightness_factor)

        # Randomly adjust contrast
        contrast_factor = random.uniform(0.5, 1.5)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(contrast_factor)

        # Resize the image to a specific size (e.g., 224x224)
        target_size = (224, 224)
        image = image.resize(target_size)

        # Apply geometric transformations (e.g., rotation)
        rotation_angle = random.uniform(-15, 15)
        image = image.rotate(rotation_angle)

        # Additional pre-processing steps can be added based on your requirements

        return image

processed_images = {}

for image_id, original_image in image_dict.items():
    processed_image = preprocess_image(original_image)
    processed_images[image_id] = processed_image

# Now, 'processed_images' contains the pre-processed images with corresponding IDs.


In [201]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from torch.autograd import Variable

# Load pre-trained ResNet model
resnet_model = models.resnet50(pretrained=True)
resnet_model.eval()

# Define a function to preprocess the image and extract features
def extract_resnet_features(image):
    # Preprocess the image
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image_tensor = preprocess(image)
    image_tensor = Variable(image_tensor.unsqueeze(0))

    # Extract features using the pre-trained ResNet model
    with torch.no_grad():
        features = resnet_model(image_tensor)

    # Remove the spatial dimensions (flatten the features)
    features = features.squeeze().numpy()

    # Normalize the features
    normalized_features = (features - features.mean()) / features.std()

    return normalized_features

extracted_features = {}

for image_id, image_path in processed_images.items():
    try:
        features = extract_resnet_features(image_path)
        extracted_features[image_id] = features
    except Exception as e:
        print(f"Error processing image with ID {image_id}: {str(e)}")



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Text feature Extraction

In [202]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Check if the text is not a string (e.g., if it's a float), then convert to string
    if not isinstance(text, str):
        text = str(text)

    # Lower-casing
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Removing punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    # Stop Word Removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming (using Porter Stemmer)
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Lemmatization (using WordNet Lemmatizer)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


preprocessed_texts = {}

for text_id, original_text in review_dict.items():
    preprocessed_text = preprocess_text(original_text)
    preprocessed_texts[text_id] = preprocessed_text

# Now, 'preprocessed_texts' contains the pre-processed texts with corresponding IDs.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [203]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = list(preprocessed_texts.values())

# Calculate TF-IDF scores
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

# Create a dictionary to store TF-IDF scores for each document
tfidf_scores = {}

# Iterate over preprocessed_texts and assign TF-IDF scores
for i, text_id in enumerate(preprocessed_texts.keys()):
    tfidf_scores[text_id] = tfidf_matrix[i].toarray()[0]

In [204]:
def get_image_and_review_by_id(dataframe, target_id):
    # Search for the entry with the specified ID
    result = dataframe[dataframe['id'] == target_id]

    # Check if any matching entry is found
    if not result.empty:
        # Extract image URL and review text
        image_url = result['Image'].tolist()  # Assuming the 'Image' column contains lists
        review_text = result['Review Text'].values[0]

        return image_url, review_text
    else:
        return None, None

IMAGE BASED RETREIVAL

In [205]:
from sklearn.metrics.pairwise import cosine_similarity
def calculate_cosine_similarity(vector1, vector2):
    # Convert arrays to NumPy arrays
    array1 = np.array(vector1).reshape(1, -1)
    array2 = np.array(vector2).reshape(1, -1)
    # Calculate cosine similarity
    similarity = cosine_similarity(array1, array2)
    # Extract the similarity score from the resulting matrix
    similarity_score = similarity[0, 0]
    return similarity_score


def find_top_related_images(input_features, extracted_features, top_n=3):
    # Calculate cosine similarity with all images in the dictionary
    similarity_scores = {}
    for image_id, features in extracted_features.items():
        similarity_scores[image_id] = calculate_cosine_similarity(input_features, features)
    # Sort images by similarity scores in descending order
    sorted_images = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)

    # Get the top N related images
    top_related_images = sorted_images[:top_n]
    return top_related_images


def find_top_related_texts(input_text, tfidf_matrix, top_n=3):
    # Calculate TF-IDF scores for the input text
    input_tfidf_vector = vectorizer.transform([input_text]).toarray()[0]
    # input_tfidf_vector = vectorizer.transform([input_text]).toarray()[0]

    # Calculate cosine similarity with all reviews
    similarity_scores = {}
    for i, review_tfidf_vector in enumerate(tfidf_matrix):
        input_tfidf_vector = np.array(input_tfidf_vector).reshape(1, -1)
        similarity = cosine_similarity(input_tfidf_vector,  review_tfidf_vector)
        similarity_score = similarity[0, 0]
        # similarity_scores[i] = calculate_cosine_similarity(input_tfidf_vector, review_tfidf_vector)
        similarity_scores[i] = similarity_score

    # Sort reviews by similarity scores in descending order
    sorted_reviews = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)

    # Get the top N related reviews
    top_related_reviews = sorted_reviews[:top_n]
    return top_related_reviews

TEXT BASED RETRIEVAL

In [208]:
image_imput_url = input("Enter image url")
review_input = input("Enter review text")
print("USING IMAGE BASED RETRIEVAL")
input_image = fetch_image(image_imput_url, 1)
input_image_features = extract_resnet_features(preprocess_image(input_image))
input_text_preprocessed = preprocess_text(review_input)

top_related_images = find_top_related_images(input_image_features, extracted_features)
# top_related_reviews = find_top_related_texts(input_text_preprocessed, tfidf_matrix)

for image_id, similarity_score in top_related_images:
    ## use image id to find image and text from the dataset.
    print(f"Image ID: {image_id}, Similarity Score: {similarity_score}")
    image_url, review_text = get_image_and_review_by_id(df, image_id)
    print(f"Image URL: {image_url}")
    print(f"Review: {review_text}")
    print(f"Cosine similarity of images: {similarity_score}")
    ## now find cosine similarity of review.
    preprocessed_text = preprocess_text(review_text)
    temp = vectorizer.transform([preprocessed_text]).toarray()[0].tolist()
    # print(temp)
    input_text_preprocessed_tfidf = vectorizer.transform([input_text_preprocessed ]).toarray()[0].tolist()
    text_similarity = calculate_cosine_similarity(input_text_preprocessed_tfidf,temp)
    print(f"Cosine similarity of text: {text_similarity}")
    print(f"Composite Similarity: {(text_similarity + similarity_score)/2}")

    print()

print("USING TEXT BASED RETRIEVAL")
## find input features.
input_image = fetch_image(image_imput_url, 1)
input_image_features = extract_resnet_features(preprocess_image(input_image))
input_text_preprocessed = preprocess_text(review_input)
top_related_text = find_top_related_texts(input_text_preprocessed, tfidf_matrix)

for image_id, similarity_score in top_related_text:
    ## use image id to find image and text from the dataset.
    print(f"Image ID: {image_id}, Similarity Score: {similarity_score}")
    image_url, review_text = get_image_and_review_by_id(df, image_id)
    print(f"Image URL: {image_url}")
    print(f"Review: {review_text}")
    print(f"Cosine similarity of text: {similarity_score}")
    input_image = fetch_image(image_imput_url, 1)
    input_image_features = extract_resnet_features(preprocess_image(input_image))
    related_image = fetch_image(image_url, 1)
    related_image_features = extract_resnet_features(preprocess_image(related_image))
    image_similarity = calculate_cosine_similarity(related_image_features, input_image_features)
    print(f"Cosine similarity of image: {image_similarity}")
    print(f"Composite similarity: {(image_similarity + similarity_score)/2}")

Enter image urlhttps://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg
Enter review textI have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.
USING IMAGE BASED RETRIEVAL
Image ID: 643, Similarity Score: 0.8881058096885681
Image URL: [['https://images-na.ssl-images-amazon.com/images/I/719-SDMiOoL._SY88.jpg']]
Review: These locking tuners look great and keep tune.  Good quality materials and construction.  Excellent upgrade to any guitar.  I had to drill additions holes for installation.  If your neck already comes with pre-drilled holes, then they should drop right in, otherwise you will need to buy a guitar tuner pin drill jig, also available from Amazon.
Cosine similarity of images: 0.8881058096885681
Cosine similarity of text: 0.132622763046278
Composite Similarity: 0.510364286367423

Image ID: 654, Similarity Score: 0.8576998710632324
Image URL: [

TypeError: Unexpected type <class 'NoneType'>